In [1]:
import numpy as np
import matplotlib
import mnist
import scipy
import scipy.sparse.linalg
from scipy.sparse import csr_matrix
from scipy.sparse import identity
import pdb
import cProfile, pstats, StringIO

In [21]:
trainX, trainY = mnist.load_mnist("training", None, './MNIST');
testX, testY = mnist.load_mnist("testing", None, './MNIST');
print("MNIST loaded")

MNIST loaded


In [3]:
#trainX = trainX[0:100, :]
#trainY = trainY[0:100]

In [4]:
class oneVsAll:
    def __init__(self, lamda):
        self.lamda = lamda;
        self.weights = {};
        
    def load_data(self, string, ori_X, ori_Y):
        self.trainY = ori_Y;
        s = ori_X.shape;
        self.d = s[1] * s[2] + 1;
        self.n = self.d;
        self.N = s[0];
        self.X = np.reshape(ori_X, (self.N, self.d-1));
        self.X = np.insert(self.X, 0, 1, axis=1);
        self.Y = np.reshape(self.trainY, (self.N, 1))
        print("Loading data " + string + " complete....")

    def makeY(self, val):
        temp = np.array(self.trainY == val, dtype=float);
        self.Y = np.reshape(temp, (self.N, 1))
        print("Classifying class " + str(self.digit))
    
    def compWeightBatch(self):
        idx_from = range(0, self.N, self.batchSize)
        idx_to = idx_from[1:]
        idx_to.extend([self.N])
        self.w = np.zeros((self.n, 1), dtype=np.float);
        for x in range(0, len(idx_from)):
            self.X = np.asmatrix(self.X);
            sigma = self.X.transpose() * self.X;
            Ident = np.identity(self.n);
            Ident = np.asmatrix(Ident);
            self.A = np.linalg.inv(sigma + self.lamda * Ident);
            self.Ax = self.A * self.X.transpose();
            self.w = self.Ax * self.Y;
        self.weights[self.digit] = self.w;
        
    def compWeightA(self):
        #pdb.set_trace()
        sigma = self.X.transpose() * self.X;
        self.A = np.linalg.inv(sigma + self.lamda * np.identity(self.n));
        self.Ax = np.dot(self.A, self.X.transpose());
        self.w = np.dot(self.Ax, self.Y);
        self.weights[self.digit] = self.w;
        
    def compWeight(self):
        #pdb.set_trace()
        self.X = np.asmatrix(self.X);
        sigma = self.X.transpose() * self.X;
        Ident = np.identity(self.n);
        Ident = np.asmatrix(Ident);
        #pdb.set_trace()
        self.A = np.linalg.inv(sigma + self.lamda * Ident);
        
        self.Ax = self.A * self.X.transpose();
        self.w = self.Ax * self.Y;
        self.weights[self.digit] = self.w;
    
    def multifit(self, batchSize):
        self.batchSize = batchSize;
        for self.digit in range(0,10):
            model.makeY(self.digit);
            model.compWeight();
            
    def predict(self):
        self.predicted = np.zeros((self.N, 10), dtype=float);
        for self.digit in range(0, 10):
            weight = self.weights[self.digit];
            self.predicted[:,self.digit] = np.reshape(np.dot(self.X, weight), self.N);
        self.predLabel = np.argmax(self.predicted, axis=1);
        
    def eval01(self):
        temp = (np.reshape(self.predLabel, (len(self.Y),1)) != np.reshape(self.Y, (len(self.Y),1))).astype(float)
        self.loss01 = np.sum(temp) / len(self.Y);
        print("0 / 1 loss : " + str(self.loss01));
    
    def squareLoss(self):
        
        lttemp = np.reshape(np.float64(self.Y==0),(self.N, 1))
        for i in range(1,10):
            lttemp = np.concatenate([lttemp, np.reshape(np.float64(self.Y==i),(self.N, 1))], axis=1);
        temp = lttemp - self.predicted
        self.sqsum = np.sum(np.square(temp))/self.N
        print("Square Loss : " + str(self.sqsum))
    
    def genGaussFeatWeight(self, n):
        self.n = n;
        self.weight = np.random.randn(self.d, self.n);
    
    def randGaussFeatConv(self):
        self.X = np.dot(self.X, self.weight);

In [5]:
model = oneVsAll(1.0);
model.load_data("training", trainX, trainY);
model.genGaussFeatWeight(10000);
model.randGaussFeatConv();
model.multifit(30000)

model.Y = model.trainY
model.predict()
model.eval01()

Loading data training complete....
Classifying class 0
Classifying class 1
Classifying class 2
Classifying class 3
Classifying class 4
Classifying class 5
Classifying class 6
Classifying class 7
Classifying class 8
Classifying class 9
0 / 1 loss : 0.89505


In [6]:
model.predict()
model.eval01()

0 / 1 loss : 0.89505


In [7]:
print('A')

A


In [8]:
model.predLabel

array([5, 0, 4, ..., 5, 6, 8])

In [9]:
model.Y = model.trainY
model.predict()
model.eval01()

0 / 1 loss : 0.142266666667


In [17]:
s = testX.shape;
model.d = s[1] * s[2] + 1;
model.n = model.d;
model.N = s[0];
model.X = np.reshape(testX, (model.N, model.d-1));
model.X = np.insert(model.X, 0, 1, axis=1);
model.Y = np.reshape(testY, (model.N, 1))
model.predict()
model.eval01()

ValueError: shapes (10000,785) and (10000,1) not aligned: 785 (dim 1) != 10000 (dim 0)

In [26]:
testX[0,:,:]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.    

In [26]:
model.predicted[0,:]

array([-0.07094919, -0.07094919,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])